In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import numpy as np
from nanomesh import Plane
from nanomesh.plane import Plane
import pyvista as pv
from skimage import filters

In [ ]:
plane = Plane.load('x500.npy')
plane = plane.crop(left=150, right=725, top=410, bottom=1470)
plane.show()

In [ ]:
plane_gauss = plane.apply(filters.gaussian, sigma=5)
# filters.try_all_threshold(plane_gauss.image, figsize=(5, 10), verbose=False)

## Local thresholding

### User-defined local threshold

In [ ]:
from skimage.filters import threshold_local
import matplotlib.pyplot as plt

offset = 150
block_size = 101

image = plane_gauss.image

local_thresh = threshold_local(image, block_size, offset=offset)
seg_local = image > local_thresh

### Otsu local threshold

In [ ]:
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
from skimage.util import img_as_ubyte

image_orig = plane_gauss.image
image_orig = (image_orig - image_orig.min()) / (image_orig.max() - image_orig.min())
image = img_as_ubyte(image_orig)

radius = 41
selem = disk(radius)

local_otsu = rank.otsu(image, selem)
seg_local_otsu = image >= local_otsu

### Compare results

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(8, 6))
ax = axes.ravel()
plt.gray()

ax[0].imshow(seg_local)
ax[0].set_title('Local thresholding')

ax[1].imshow(local_otsu, cmap=plt.cm.gray)
ax[1].set_title('Local Otsu (radius=%d)' % radius)
ax[1].axis('off')

ax[2].imshow(seg_local_otsu, cmap=plt.cm.gray)
ax[2].set_title('Original >= Local Otsu')
ax[2].axis('off')

for a in ax:
    a.axis('off')

plt.show()

In [ ]:
from skimage.segmentation import clear_border

# select subsection for faster processing
sub_seg = seg_local[20:110, 40:220]

# seg = (~clear_border(~sub_seg)).astype(int)
seg = sub_seg

plt.close('all')
plt.imshow(seg)
plt.show()

## Contour finding

In [ ]:
%%time

from nanomesh.mesh2d import Mesher2D

mesher = Mesher2D(seg)

mesher.add_points(label=1)
mesher.add_points(label=0, point_density=0.02)
mesher.generate_contours(max_contour_dist=4)
mesher.generate_edge_contours(max_contour_dist=10)
mesher.generate_mesh()
mesher.generate_domain_mask()

mesher.plot_steps()